In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
import re


In [2]:
from extract_dataframe import read_json         # a function to load json_data 
from extract_dataframe import TweetDfExtractor  # and a class to extract relevant variables.    
from clean_tweets_dataframe import Clean_Tweets        # collection of functions to for cleaning the dataframe.


load json 

In [3]:
_, tweet_list = read_json("data/africa_twitter_data.json")
tweet = TweetDfExtractor(tweet_list)
tweet.get_tweet_df(True)  # this will also generate a CSV file.
print("Please, load the CSV file")

File Successfully Saved.!!!
Please, load the CSV file


Load the CSV

In [4]:
tweet_df = pd.read_csv('processed_tweet_data.csv')  # loads the csv file created above.


In [6]:
tweet_df.shape

(222, 17)

In [7]:
cleaner = Clean_Tweets(tweet_df)
tweet_df = cleaner.drop_unwanted_column(tweet_df)
tweet_df = cleaner.drop_duplicate(tweet_df)
tweet_df = cleaner.convert_to_datetime(tweet_df)
tweet_df = cleaner.convert_to_numbers(tweet_df)
clean_tweet_df = cleaner.remove_non_english_tweets(tweet_df)


Automation in Action...!!!
Duplicate rows successfully removed


In [8]:
clean_tweet_df.shape

(222, 17)

In [9]:
clean_tweet_df.describe() 

,polarity,subjectivity,favorite_count,retweet_count,followers_count,friends_count
count,222.0,222.0,222.000000,222.000000,222.000000,222.000000
mean,0.0,0.0,8443.022523,1.639640,6408.864865,1399.135135
std,0.0,0.0,20149.243557,7.147816,65959.680158,7048.253677
min,0.0,0.0,0.000000,0.000000,0.000000,1.000000
25%,0.0,0.0,284.000000,0.000000,97.000000,158.500000
50%,0.0,0.0,1070.500000,0.000000,351.000000,382.000000
75%,0.0,0.0,4209.250000,1.000000,1107.250000,954.000000
max,0.0,0.0,158375.000000,91.000000,977705.000000,103722.000000


In [10]:
# selecting only the relevant variables for further analysis
sho_tweet_df = clean_tweet_df.drop(['original_text', 'user_mentions', 'possibly_sensitive'], axis=1)
loc_tweet_df = sho_tweet_df.dropna() # droping rows with null values

#resetting indexes
sho_tweet_df = sho_tweet_df.reset_index();
sho_tweet_df = sho_tweet_df.drop(columns=['index'])

loc_tweet_df = loc_tweet_df.reset_index();
loc_tweet_df = loc_tweet_df.drop(columns=['index'])

print(clean_tweet_df.shape)
print(sho_tweet_df.shape)
print(loc_tweet_df.shape)

(222, 17)
(222, 14)
(218, 14)


In [11]:
def sentiment_group (p): 
    if p > 0:
        return 'positive'
    elif p < 0:
        return 'negative'
    else:
        return 'neutral'

In [12]:
sentiment = pd.Series([sentiment_group(row_val) for row_val in list(loc_tweet_df['polarity'])])


In [13]:
sho_tweet_df = pd.concat([sho_tweet_df, sentiment.rename("sentiment")], axis=1)
loc_tweet_df = pd.concat([loc_tweet_df, sentiment.rename("sentiment")], axis=1)

sho_tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   created_at       222 non-null    datetime64[ns, UTC]
 1   source           222 non-null    object             
 2   clean_text       222 non-null    object             
 3   polarity         222 non-null    float64            
 4   subjectivity     222 non-null    float64            
 5   lang             222 non-null    object             
 6   favorite_count   222 non-null    int64              
 7   retweet_count    222 non-null    int64              
 8   screen_name      222 non-null    object             
 9   original_author  222 non-null    object             
 10  followers_count  222 non-null    int64              
 11  friends_count    222 non-null    int64              
 12  hashtags         222 non-null    object             
 13  place            218